<a href="https://colab.research.google.com/github/hail-members/llm-based-services/blob/main/Chapter_5_%EC%8B%A4%EC%8A%B5%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Train 데이터 다운로드
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_train.json -O KorQuAD_v1.0_train.json

# Dev 데이터 다운로드
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json -O KorQuAD_v1.0_dev.json

import json

# Train 데이터 로드
with open("KorQuAD_v1.0_train.json", "r", encoding="utf-8") as f:
    train_data = json.load(f)

# Dev 데이터 로드
with open("KorQuAD_v1.0_dev.json", "r", encoding="utf-8") as f:
    dev_data = json.load(f)

# 데이터 구조 확인
print("Train Data Keys:", train_data.keys())
print("Example Data:", train_data["data"][0])  # 첫 번째 문단 출력

--2025-04-04 10:52:23--  https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.111.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38527475 (37M) [application/json]
Saving to: ‘KorQuAD_v1.0_train.json’

KorQuAD_v1.0_train. 100%[===================>]  36.74M  11.1MB/s    in 3.3s    

2025-04-04 10:52:28 (11.1 MB/s) - ‘KorQuAD_v1.0_train.json’ saved [38527475/38527475]

--2025-04-04 10:52:29--  https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
Resolving korquad.github.io (korquad.github.io)... 185.199.110.153, 185.199.111.153, 185.199.108.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3881058 (3.7M) [application/json]
Saving to: ‘KorQuAD_v1.0_dev.json’

KorQuAD_v1.0_dev.js 100%[=====

In [2]:
# 라이브러리 임포트
import torch
from transformers import GPT2LMHeadModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader


# KoGPT2 모델과 토크나이저 로드
model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    bos_token='</s>',
    eos_token='</s>',
    unk_token='<unk>',
    pad_token='<pad>',
    mask_token='<mask>'
)
tokenizer.add_special_tokens({'additional_special_tokens': ['<usr>', '<sys>']})
model = GPT2LMHeadModel.from_pretrained(model_name)

In [3]:
# 데이터셋 정의 (질문과 답변을 GPT2 입력 형식으로 변환)
class KorQuADDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=50):
        self.data = data["data"]
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.total_qas = []  # 모든 질문-답변 쌍을 저장

        # 전체 질문-답변 쌍을 리스트로 저장
        for article in self.data:
            for paragraph in article["paragraphs"]:
                for qa in paragraph["qas"]:
                    self.total_qas.append((paragraph["context"], qa))

    def __len__(self):
        return len(self.total_qas)

    # def __getitem__(self, idx):
    #     # 질문과 답변 추출
    #     context, qa = self.total_qas[idx]
    #     question = qa["question"]
    #     answer_text = qa["answers"][0]["text"]

    #     # GPT2 입력 텍스트 생성 (질문 + 컨텍스트 + 답변)
    #     # input_text = f"<usr> {question} <sys> {context} </s>"
    #     # GPT2 입력 텍스트 생성 (질문+ 답변)
    #     input_text = f"<usr> {question} </s>"
    #     target_text = f"<sys> {answer_text} </s>"

    #     # 토큰화 및 인코딩
    #     input_ids = self.tokenizer.encode(input_text, max_length=self.max_length, truncation=True, padding="max_length")
    #     target_ids = self.tokenizer.encode(target_text, max_length=self.max_length, truncation=True, padding="max_length")

    #     return {
    #         "input_ids": torch.tensor(input_ids),
    #         "labels": torch.tensor(target_ids)
    #     }
    def __getitem__(self, idx):
        context, qa = self.total_qas[idx]
        question = qa["question"]
        answer = qa["answers"][0]["text"]

        # 전체 시퀀스 생성 (질문+답변)
        full_text = f"<usr> {question} </s> <sys> {answer} </s>"
        
        # 통합 토큰화
        encoding = self.tokenizer(
            full_text,
            max_length=self.max_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        # 레이블 마스킹 처리
        input_ids = encoding.input_ids.squeeze()
        labels = input_ids.clone()

        # 질문 부분 마스킹 (</s> 토큰 직전까지)
        sep_token = self.tokenizer.convert_tokens_to_ids("</s>")
        sep_positions = (input_ids == sep_token).nonzero(as_tuple=True)[0]
        if len(sep_positions) >= 1:
            labels[:sep_positions[0]+1] = -100  # 질문 부분 무시

        return {
            "input_ids": input_ids,
            "labels": labels
        }


train_dataset = KorQuADDataset(train_data, tokenizer)
dev_dataset = KorQuADDataset(dev_data, tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
dev_dataloader = DataLoader(dev_dataset, batch_size=4)


In [4]:
# 저장된 데이터 보기
for batch in train_dataloader:
    print(batch)
    print(tokenizer.decode(batch["input_ids"][0], skip_special_tokens=False))
    break

{'input_ids': tensor([[    2, 32341,  7426,  9489,  9060, 17823, 19344, 24204,  9717, 22375,
         12619,  9057,  7055,   406,   739,     1,   739,     4,  9097, 12287,
         15215,   739,     1,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3],
        [    2, 40391,  8042,  9123, 10583, 13400,  9023,  9981,  7689,  8470,
          8137, 10648, 17161,   406,   739,     1,   739,     4, 10592,   409,
          9563,   739,     1,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3],
        [    2, 10364,  9382,  9439, 23113, 39560,   406,   739,     1,   739,
             4, 10364, 11696, 10352,   739,     1,     3,     3,     3,     3,
             3,     3,     3,     3,

In [5]:
# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 문장 생성 함수 정의
def generate_sentence(model, seed_text, max_length=50):
    input_ids = tokenizer.encode(seed_text, return_tensors="pt").to(device)
    gen_ids = model.generate(
        input_ids,
        max_length=max_length,
        repetition_penalty=2.0,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
        use_cache=True,
    )
    generated_text = tokenizer.decode(gen_ids[0], skip_special_tokens=True)
    return generated_text

question = "인공지능이란?"
context = ""

# 입력 텍스트 생성
input_text = f"<usr> {question} <sys> {context} </s>"
input_ids = tokenizer.encode(
    input_text,
    max_length=100,
    truncation=True,
    padding="max_length",
    return_tensors="pt"
).to(model.device)

# KoGPT2로 문장 생성
model.eval()
with torch.no_grad():
    output_ids = model.generate(
        input_ids=input_ids,
        max_length=150,
        repetition_penalty=2.0,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
    )

generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# 결과 출력
print(f"질문: {question}")
print(f"컨텍스트: {context}")
print(f"생성된 답변: {generated_text}")


질문: 인공지능이란?
컨텍스트: 
생성된 답변: 인공지능이란?   
, , . (중략) 이번에 출시된 신제품은 '스마트폰용 스마트패드'다.
이 제품은 스마트폰을 통해 다양한 콘텐츠를 즐길 수 있는 것이 특징이다.
특히 기존 제품보다 최대 2배 이상 빠른 속도로 데이터를 전송


In [ ]:

from torch.optim import AdamW
from tqdm import tqdm

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=5e-3)

# 학습 루프 정의 (tqdm으로 Progress Bar 추가)
epochs = 10
model.train()

# 조기 종료 조건 설정
# max_batches_per_epoch = 1000  # 한 에포크에서 최대 실행할 배치 수

for epoch in range(epochs):
    epoch_loss = 0
    progress_bar = tqdm(enumerate(train_dataloader), desc=f"Epoch {epoch + 1}", total=len(train_dataloader))

    for batch_idx, batch in progress_bar:
        # if batch_idx >= max_batches_per_epoch:  # 조기 종료 조건
        #     print(f"Stopping early at batch {batch_idx} in epoch {epoch + 1}")
        #     break

        input_ids = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss
        print(f"질문: {tokenizer.decode(input_ids[0], skip_special_tokens=False)}")

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        progress_bar.set_postfix({"Batch Loss": loss.item()})

    print(f"Epoch {epoch + 1} completed. Average Loss: {epoch_loss / (batch_idx + 1)}")

# 모델 저장
model.save_pretrained("./kogpt2-korquad-finetuned")
tokenizer.save_pretrained("./kogpt2-korquad-finetuned")

Epoch 1:   0%|          | 0/15102 [00:00<?, ?it/s]

질문: <usr> 독립선언 이전의 영국 식민지들은 영국의 왕권에 대하여 어떤 의무를 실행해야 했는가? </s> <sys> 충성의 의무 </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


Epoch 1:   0%|          | 3/15102 [00:00<33:43,  7.46it/s, Batch Loss=1.28]   

질문: <usr> 김정은의 성형수술설의 보도경위와 근거를 추적한 매체는? </s> <sys> 신화통신 </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
질문: <usr> 멩겔레가 유태인의 인종적 차이점에 대한 논문을 써서 인류학 박사 학위를 받은 년도는? </s> <sys> 1935 </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
질문: <usr> 러스티의 논문은 어떤 사람의 이론의 기반이 되었나? </s> <sys> 파스퇴르 </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
질문: <usr> 기원전 1세기 중반에 율리우스 카이사르와 폼페이우스 마그누스, 크라수스 세 사람이 공화국을 농단했던 정치는? </s> <sys> 삼두정치 </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
질문: <usr> Ac와 섞여서 뼈에 축적되는 악티늄을 줄이는데 효과가 있는 물질은? </s> <sys> 시트르산염, 에틸렌다이아민테트라아세트산(EDTA), 다이에틸렌트라이아민펜타아


Epoch 1:   0%|          | 9/15102 [00:00<15:50, 15.87it/s, Batch Loss=1.18] 

...

Epoch 1:   1%|          | 156/15102 [00:07<10:27, 23.80it/s, Batch Loss=0.767]

질문: <usr> 헤이우드는 누구의 방대한 저술들의 핵심을 단순한 명제로 증류시켰는가? </s> <sys> 카를 마르크스 </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
질문: <usr> 폴 로다는 어느나라 사람인가? </s> <sys> 영국 </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
질문: <usr> 국회의사당이 폭파 될 때 불꽃놀이의 모양은 무엇인가? </s> <sys> V 모양 </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
질문: <usr> 대한민국에 민주주의가 들어서기 시작한 년도는? </s> <sys> 1987년 </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
질문: <usr> 고려시대 광종이 혜거국사를 왕사로 모시러 갔다가 마시고 속병이 나았다고 하는 어정수가  나오는 절 이름은? </s> <sys> 칠성암 </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

Epoch 1:   1%|          | 162/15102 [00:07<10:34, 23.54it/s, Batch Loss=0.845]

질문: <usr> 전리수소영역에서 태어난 별들 중 가장 무거운 것들이 일으키는 것은? </s> <sys> 초신성 폭발 </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
질문: <usr> 전시 국무 차관이자 미육군 공병사령부의 지휘권을 갖고 있던 사람은 누구인가? </s> <sys> 로버트 페터슨 </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
질문: <usr> 태풍 매미로 우리나라에서 측정된 최대순간풍속에서 상위를 기록한 대부분의 지방이 위치한 곳은? </s> <sys> 남부 </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
질문: <usr> 폴링은 벤젠의 구조를 무엇에 기초하여 설명하였는가 </s> <sys> 양자역학 </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
질문: <usr> 맥그리거의 딸은 몇 명인가? </s> <sys> 4명 </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


Epoch 1:   1%|          | 162/15102 [00:07<10:34, 23.54it/s, Batch Loss=0.557]

In [7]:

question = "인공지능이란?"

# 입력 텍스트 생성
input_text = f"<usr> {question} </s>"
input_ids = tokenizer.encode(
    input_text,
    max_length=100,
    truncation=True,
    padding="max_length",
    return_tensors="pt"
).to(model.device)

# KoGPT2로 문장 생성
model.eval()
with torch.no_grad():
    output_ids = model.generate(
        input_ids=input_ids,
        max_length=400,
        repetition_penalty=2.0,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
        bos_token_id=tokenizer.bos_token_id,
    )

generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=False)

# 결과 출력
print(f"질문: {question}")
print(f"생성된 답변: {generated_text}")

질문: 인공지능이란?
생성된 답변: <usr> 인공지능이란? </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad> <sys> </s>


In [8]:
model.eval()
for epoch in range(epochs):
    epoch_loss = 0
    progress_bar = tqdm(enumerate(train_dataloader), desc=f"Epoch {epoch + 1}", total=len(train_dataloader))

    for batch_idx, batch in progress_bar:
        # if batch_idx >= max_batches_per_epoch:  # 조기 종료 조건
        #     print(f"Stopping early at batch {batch_idx} in epoch {epoch + 1}")
        #     break

        input_ids = batch["input_ids"].to(device)
        # cut off where the question endss
        # find where the question ends
        sep_token = tokenizer.convert_tokens_to_ids("<sys>")
        sep_positions = (input_ids == sep_token).nonzero(as_tuple=True)[1]
        if len(sep_positions) >= 1:
            input_ids = input_ids[:,:sep_positions[0]+1]
        
        
        labels = batch["labels"].to(device)

        with torch.no_grad():
            output_ids = model.generate(
                input_ids=input_ids,
                max_length=513,
                repetition_penalty=2.0,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                bos_token_id=tokenizer.bos_token_id,
            )
        
        print(f"질문: {tokenizer.decode(input_ids[0], skip_special_tokens=False)}")
        print(f"생성된 답변: {tokenizer.decode(output_ids[0], skip_special_tokens=False)}")
        break

Epoch 1:   0%|          | 0/15102 [00:04<?, ?it/s]

질문: <usr> 세 가지 형태의 환경교육 방법들이 별 의미가 없다고 하여 통합하여 부르는 용어는? </s> <sys>
생성된 답변: <usr> 세 가지 형태의 환경교육 방법들이 별 의미가 없다고 하여 통합하여 부르는 용어는? </s> <sys> <pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p